## Evaluate and monitor a model online



In [1]:
# runs in jupyter container on node-eval-online-online
import os, base64, time, random, requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


### Monitor operational metrics

To monitor a service using Prometheus, it needs to expose an HTTP endpoint (by convention, `/metrics`) that Prometheus can scrape. For standard frameworks like Flask or FastAPI (or Triton inference server, for that matter) it is easy to use existing libraries to quickly instrument a service.

We have already started an [updated FastAPI service](https://github.com/teaching-on-testbeds/eval-online-chi/tree/main/fastapi_pt), which looks very similar to one we’ve developed previously, but now:

-   in `requirements.txt`, we’ve added the `prometheus-fastapi-instrumentator` library as a dependency
-   and inside [the app itself](https://github.com/teaching-on-testbeds/eval-online-chi/blob/main/fastapi_pt/app.py), we’ve added this to the beginning:

``` python
from prometheus_fastapi_instrumentator import Instrumentator
```

-   and this to the end:

``` python
Instrumentator().instrument(app).expose(app)
```

Meanwhile, we have configured our Prometheus service to get metrics from this service every 15 seconds, in `prometheus.yml`:

    global:
      scrape_interval: 15s

    scrape_configs:
      - job_name: 'food11'
        static_configs:
          - targets: ['fastapi_server:8000']

That’s all it takes to start sharing the basic operational metrics of a FastAPI service!

Now, if we visit

    http://A.B.C.D:8000/docs

in a browser, but using the floating IP assigned to our instance in place of `A.B.C.D`, we’ll note that in addition to a `/predict` endpoint, our FastAPI service now has a `/metrics` endpoint. This is the endpoint that will be scraped by Prometheus.

Click on “/metrics”, then “Try it out” and “Execute” to see what it returns.

Before we look at the actual metrics in Prometheus, let’s generate some data from operational use.

In a browser, we’ll visit

    http://A.B.C.D:5000

but using the floating IP assigned to our instance in place of `A.B.C.D`. Then, upload a few images to the web app and submit them for classification.

Now (or, within 15 seconds!) we can look up some operational metrics related to those queries in Prometheus.

In a browser, we’ll visit

    http://A.B.C.D:9090

but using the floating IP assigned to our instance in place of `A.B.C.D`. This will open the Prometheus web UI.

First, let’s find out about the services that this Prometheus instance is monitoring. From the menu, choose “Status” \> “Targets” and note the FastAPI endpoint and its status.

Next, let’s look at some of the metrics data. Prometheus uses a query language called PromQL to query metrics, but we can find some metrics of interest without knowing any PromQL. From the menu, choose “Graph”. Then, click on the ⋮ on the right side the query bar and choose “Explore metrics” to see some of the available metrics we can query. Find “http_requests_total” in the list and click on it to select it, and click “Execute”.

Prometheus will show us metrics related to all endpoints of the FastAPI service, but we are primarily interested in the metrics associated with the “/predict” endpoint. Copy the line

    http_requests_total{handler="/predict", instance="fastapi_server:8000", job="food11", method="POST", status="2xx"}

and paste *that* into the query bar. Hit Execute.

Switch from the “Table” tab to the “Graph” tab to see a visualization of the cumulative number of requests served by this endpoint. Revisit the GourmetGram app again, and upload a few more images for classification; then execute the query in Prometheus again (or, after 15 seconds!) and confirm that it is incremented accordingly.

Instead of getting the cumulative count of requests, we may prefer to see the rate of requests per second. Put

    rate(http_requests_total{handler="/predict", instance="fastapi_server:8000", job="food11", method="POST", status="2xx"}[1m])

into the query bar and click “Execute” to see the rate of requests per second, averaged over a 1 minute moving window. You can use the + and - buttons to adjust the range of the horizontal axis to the time interval for which you have measurements.

To see more, let’s generate some more requests. We’ll send some requests directly from this notebook to the FastAPI service.

In [6]:
# runs in jupyter container on node-eval-online-online
image_path = "apple2.jpg"
with open(image_path, 'rb') as f:
    image_bytes = f.read()
encoded_str =  base64.b64encode(image_bytes).decode("utf-8")

In [7]:
# runs in jupyter container on node-eval-online
FASTAPI_URL = "http://192.5.86.161:8500/predict"
payload = {"image": encoded_str}
num_requests = 100

for _ in range(num_requests):
    response = requests.post(FASTAPI_URL, json=payload)

In [15]:
import requests

FASTAPI_URL = "http://192.5.86.161:8500/predict/"
image_path = "apple2.jpg"

for i in range(5):
    with open(image_path, "rb") as f:
        files = {"file": f}
        response = requests.post(FASTAPI_URL, files=files)
        print(f"{i+1:02d}: {response.status_code}")


01: 200
02: 200
03: 200
04: 200
05: 200


In [ ]:
image_path = "apple2.jpg"
FASTAPI_URL = "http://192.5.86.161:8500/predict/"

with open(image_path, "rb") as f:
    files = {"file": f}  
    response = requests.post(FASTAPI_URL, files=files)

print("Status:", response.status_code)
print("Body:", response.text)


Now that we have generated some data that we can query, let’s use Prometheus to understand how long it takes our inference endpoint to serve requests.

Request durations are reported as histograms, which is a slightly more involved metric type. Use the query:

    rate(http_request_duration_seconds_sum{handler="/predict", job="food11"}[1m]) / 
    rate(http_request_duration_seconds_count{handler="/predict", job="food11"}[1m])

to see the average duration of HTTP requests averaged over a 1 minute window.

Averages can be skewed, though - so let’s also look at the median, using the query:

    histogram_quantile(0.5, rate(http_request_duration_seconds_bucket{handler="/predict", job="food11"}[1m]))

or, to get a sense of the worst-case user experience, we can check the 95th percentile using the query:

    histogram_quantile(0.95, rate(http_request_duration_seconds_bucket{handler="/predict", job="food11"}[1m]))

For ongoing monitoring, we can create a dashboard using Grafana. In a browser, we’ll visit

    http://A.B.C.D:3000

but using the floating IP assigned to our instance in place of `A.B.C.D`. This will open the Grafana web UI.

Sign in using the username `admin` and password `admin`. You will be prompted to change the initial password.

Then, you’ll configure it to connect to Prometheus:

-   From the menu, choose: “Connections” \> “Data Sources”.
-   Click “Add data source” and choose “Prometheus”, then “Add new data source”.
-   Set URL to http://prometheus:9090.
-   Click “Save & Test”

Next, we’re going to build a dashboard!

-   Click “Dashboards” \> “Create dashboard” and “Add visualization”.
-   Select your Prometheus data source.
-   Near the bottom, you will see the query builder for your Prometheus data source. Click the “Code” button to toggle from “builder mode” to “code (PromQL) mode”.
-   Put the following query in the box and then click “Run queries”:

<!-- -->

    1000 * rate(http_request_duration_seconds_sum{handler="/predict", job="food11"}[1m]) / 
    rate(http_request_duration_seconds_count{handler="/predict", job="food11"}[1m])

-   At the top of the graph, adjust the visible time range to 15 minutes, so that you can see the data
-   On the right side, change the panel options: Set title to “Food11 average request duration (ms)”
-   When you are satisfied with the appearance, click “Save dashboard” and give your dashboard the title “Food11 Service Monitoring”

Now, you can click on “Dashboards” \> “Food11 Service Monitoring” to see your dashboard.

Let’s add another panel, showing the median, 95th percentile, and 99th percentile request duration.

-   Open the dashboard, and click “Edit” in the top right. Then click “Add” \> “Visualization”.
-   In the query editor, paste this query and “Run queries”:

<!-- -->

    1000*histogram_quantile(0.5, rate(http_request_duration_seconds_bucket{handler="/predict"}[1m]))

-   Then, click “Add query” and repeat for this query:

<!-- -->

    1000*histogram_quantile(0.95, rate(http_request_duration_seconds_bucket{handler="/predict"}[1m]))

-   Finally, click “Add query” and add the third query:

<!-- -->

    1000*histogram_quantile(0.99, rate(http_request_duration_seconds_bucket{handler="/predict"}[1m]))

-   Click on “Legend” for each query, set it to “Custom”, and name each series appropriately (e.g. “Median”, “95th percentile”, “99th percentile)
-   On the right side, change the panel options: Set title to “Food11 request duration percentiles (ms)”
-   At the top of the graph, adjust the visible time range to 15 minutes, so that you can see the data
-   When you are satisfied with the appearance, click “Save dashboard” to save your changes.

Return to the dashboard. You can resize panels and drag them to place them in different positions for better visibility.

Repeat the process to add additional panels to your dashboard:

-   Requests per second:

<!-- -->

    rate(http_requests_total{handler="/predict"}[1m])

-   and requests that return an error status or fail (this will have “no data” for now). You’ll add two queries, one for error statuses and one for failed requests:

<!-- -->

    # set this up as the first query
    rate(http_requests_total{handler="/predict", status="4xx"}[1m])

    # set this up as a second query in the same visualization
    rate(http_requests_total{handler="/predict", status="5xx"}[1m])

Return to the dashboard. You can resize panels and drag them to place them in different positions for better visibility.

Now, let’s generate variable load - we will ramp up, then down, the load on the FastAPI service. While the cell below is running, observe the effect on your Grafana dashboard (you can set the dashboard to auto refresh with a frequency of 5 seconds, to make it easier to monitor!):

In [16]:
import time
import requests
from concurrent.futures import ThreadPoolExecutor

FASTAPI_URL = "http://192.5.86.161:8500/predict/"
IMAGE_PATH = "apple2.jpg"

load_pattern = [1, 2, 3, 5, 3, 2, 1]  # 并发线程数的变化
delay_between_steps = 30  # 每阶段持续秒数

def send_continuous_requests(duration_sec):
    start_time = time.time()
    while time.time() - start_time < duration_sec:
        with open(IMAGE_PATH, "rb") as f:
            files = {"file": f}
            try:
                response = requests.post(FASTAPI_URL, files=files, timeout=5)
                print(f"Status: {response.status_code}")
            except Exception as e:
                print(f"Error: {e}")

def run_load_stage(concurrent_workers, duration_sec):
    print(f"🚀 Load stage: {concurrent_workers} threads for {duration_sec} seconds")
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [executor.submit(send_continuous_requests, duration_sec) for _ in range(concurrent_workers)]
        for f in futures:
            f.result()


for load in load_pattern:
    run_load_stage(load, delay_between_steps)


🚀 Load stage: 1 threads for 30 seconds
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
🚀 Load stage: 2 threads for 30 seconds
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
🚀 Load stage: 3 threads for 30 seconds
Status: 200
Status: 200
Error: HTTPConnectionPool(host='192.5.86.161', port=8500): Read timed out. (read timeout=5)
Status: 200
Error: HTTPConnectionPool(host='192.5.86.161', port=8500): Read timed out. (read timeout=5)
Error: HTTPConnectionPool(host='192.5.86.161', port=8500): Read timed out. (read timeout=5)
Error: HTTPConnectionPool(host='192.5.86.161', port=8500): Read timed out. (read timeout=5)
Error: HTTPConnectionPool(host='192.5.86.161', 

When it is done, take a screenshot of the “Food11 Service Monitoring” dashboard for later reference.

We can also generate some error responses, to see them on our dashboard. Visit

    http://A.B.C.D:8000/docs

in a browser, but using the floating IP assigned to our instance in place of `A.B.C.D`. Click on the “predict” endpoint, and “Try it out”. Set the request body to

    { }

and click “Execute”. Scroll down to see the server response, which will be

-   Code: 422
-   Details: Error: Unprocessable Entity

### Alert on operational metrics

Grafana and/or Prometheus can also be configured with alerting - they can email, post a message in Slack, or execute various other notification actions based on operational metrics. As a demo, let’s configure Grafana to alert when there is more than 20 error or failed responses/second for at least 1 minute.

On your dashboard, click on the ⫶ in the “Error responses” panel, and under “More”, choose “New Alert Rule”.

-   In “1. Enter alert rule name”, name it “Error responses”
-   In “2. Define query and alert condition”:
    -   Delete the existing “C” and “D” expressions using the trash icon.
    -   Click “Add expression” \> “Reduce”. Under “Expressions” \> “C Reduce”, choose input A, function Last, and mode Replace non-numeric values with 0.
    -   Click “Add expression” \> “Reduce”. Under “Expressions” \> “D Reduce”, choose input B, function Last, and mode Replace non-numeric values with 0.
    -   Click “Add expression” \> “Math”. Under “Expressions” \> “E Math”, use: `$C + $D` to get the sum of the most recent values of A and B
    -   Click “Add expression” \> “Threshold”. Under “Expressions” \> “F Threshold”, configure it to alert when input “E” (the “math” expression) is above 20. Click “Set F as alert condition”.
    -   Click “Preview” to verify that no errors are raised.
-   In “3. Add folder and labels”, create a new folder named “Food11”
-   In “4. Set evaluation behavior”, create a new evaluation group named “Food11” with a 30s evaluation interval. Set the “Pending period” to 1m.
-   In “5. Configure notifications”, choose to “View or create contact points”. Since we have not configured this Grafana instance to actually send emails, we’re going to create a “No-op” contact just to observe alerts within Grafana itself. Click “Create contact point”, name it “No-op”, select the “Webhook” integration, and put “http://localhost/” in the URL field. Save the contact point. Then, on the alert rule setup page, click the refresh icon next to the “Contact point” drop-down menu, and choose the “No-op” option
-   Click “Save rule and exit” in the top right.

Now, if you click on “Alerting” \> “Alert rules”, you should see a “Food11” folder with a “Food11” group inside it, and in it, your rule on “Error responses”.

Let’s generate some error responses to trigger this alert. We’re executing a similar “ramp-up/ramp-down” request pattern as before, but sending either

-   an empty payload, which we know will make the server return a 422 code
-   or an invalid payload, which will make the server return a 500 code

(instead of 200, which is a code indicating success).

In [17]:
import time
import random
import requests
from concurrent.futures import ThreadPoolExecutor

# FastAPI service running in Docker Compose with internal name
FASTAPI_URL = "http://192.5.86.161:8500/predict/"

# Load pattern: number of concurrent threads per stage
load_pattern = [1, 3, 5, 10, 5, 3, 1]
delay_between_steps = 10  # seconds per stage

def send_continuous_requests(duration_sec):
    """
    Continuously sends requests for a given duration.
    Sends either an empty or malformed payload to trigger 4xx or 5xx responses.
    """
    start_time = time.time()
    while time.time() - start_time < duration_sec:
        payload = random.choice([{}, {"image": "not_base64_data"}])
        try:
            response = requests.post(FASTAPI_URL, json=payload, timeout=5)
            print("Status:", response.status_code)
        except Exception as e:
            print("Error:", e)

def run_load_stage(concurrent_workers, duration_sec):
    """
    Runs multiple threads to send load in parallel.
    """
    print(f"🚀 Load stage: {concurrent_workers} threads for {duration_sec} seconds")
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [executor.submit(send_continuous_requests, duration_sec) for _ in range(concurrent_workers)]
        for f in futures:
            f.result()  # wait for all threads

# Execute each stage of the load pattern
for load in load_pattern:
    run_load_stage(load, delay_between_steps)


🚀 Load stage: 1 threads for 30 seconds
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
Status: 422
S

In [19]:
import time
import base64
import requests
from concurrent.futures import ThreadPoolExecutor

# URL of the FastAPI server (reachable via Docker network hostname)
FASTAPI_URL = "http://fastapi_server:8000/predict"

# Load and encode the image file to base64
image_path = "apple2.jpg"
with open(image_path, "rb") as f:
    encoded_str = base64.b64encode(f.read()).decode("utf-8")

# JSON payload to send in the POST request
payload = {"image": encoded_str}

# Load pattern: number of concurrent threads for each stage
load_pattern = [1, 2, 3, 5, 3, 2, 1]
# Duration (in seconds) for each load stage
delay_between_steps = 60

# Function to send requests continuously for a given duration
def send_continuous_requests(duration_sec):
    start_time = time.time()
    while time.time() - start_time < duration_sec:
        try:
            response = requests.post(FASTAPI_URL, json=payload, timeout=5)
            print("Status:", response.status_code)
        except Exception as e:
            print("Error:", e)

# Function to run a load stage with the specified number of concurrent threads
def run_load_stage(concurrent_workers, duration_sec):
    print(f"🚀 Load stage: {concurrent_workers} threads for {duration_sec} seconds")
    with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [
            executor.submit(send_continuous_requests, duration_sec)
            for _ in range(concurrent_workers)
        ]
        for f in futures:
            f.result()

# Run the load stages sequentially
for load in load_pattern:
    run_load_stage(load, delay_between_steps)


🚀 Load stage: 1 threads for 60 seconds
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
S

Let this run for about ten minutes. Then, come back a few minutes later (because in this dashboard, rate metrics are over a 10 minute window) and take a screenshot of the “cAdvisor Docker Insights” dashboard for later reference. (You don’t have to capture the entire dashboard, it’s fine if it’s just the top panels.)